In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
#Import Dataset
dataset=pd.read_csv("Swaps_Nominal.csv")
dataset.head()

,Date,1 year,2 year,3 year,4 year,5 year,6 year,7 years,8 years,9 years,10 years,15 year,20 years
0,5/12/11,5.46,5.545,5.66,5.725,5.790,5.89,5.855,5.885,5.920,5.95,5.97,6.00
1,5/13/11,5.59,5.625,5.70,5.735,5.773,5.84,5.855,5.885,5.920,5.95,5.97,6.00
2,5/16/11,5.59,5.620,5.69,5.717,5.750,5.83,5.835,5.870,5.915,5.93,5.95,5.98
3,5/17/11,5.60,5.640,5.68,5.703,5.720,5.82,5.805,5.855,5.905,5.94,5.96,5.99
4,5/18/11,5.63,5.650,5.69,5.710,5.720,5.80,5.800,5.857,5.930,5.96,5.98,6.01


In [7]:
'''
dataset=pd.read_csv("SWAPRATES_PYTHON.csv")
dataset.head()
dataset.shape
'''
dataset=dataset.dropna()
pd.set_option('display.max_columns', None)

dataset.head()
dataset.shape
#dataset.set_index('Date', inplace=True)
dataset=dataset.iloc[:,1:12]
dataset.head()


,1 year,2 year,3 year,4 year,5 year,6 year,7 years,8 years,9 years,10 years,15 year
0,5.46,5.545,5.66,5.725,5.790,5.89,5.855,5.885,5.920,5.95,5.97
1,5.59,5.625,5.70,5.735,5.773,5.84,5.855,5.885,5.920,5.95,5.97
2,5.59,5.620,5.69,5.717,5.750,5.83,5.835,5.870,5.915,5.93,5.95
3,5.60,5.640,5.68,5.703,5.720,5.82,5.805,5.855,5.905,5.94,5.96
4,5.63,5.650,5.69,5.710,5.720,5.80,5.800,5.857,5.930,5.96,5.98


In [17]:
#RUN PCA LOOP using residuals to predict if residuals go down in 7 days
i=0
j=50
m=50
n=58
window_size=50
n_loops=dataset.shape[0]/window_size
pos_count_list0=[]
neg_count_list=[]

while i <=dataset.shape[0]:
    x_c=dataset.iloc[i:j,0:12].values
    x_m=dataset.iloc[m:n,0:12].values
    j=j+50
    i=i+50
    m=m+50
    n=n+50
    covariance_matrix=np.cov(x_c.T)
    eigen_values,eigen_vectors=np.linalg.eig(covariance_matrix)    #eigenvalues/vectors
    
    Projection_Matrix=(eigen_vectors.T[:][:])[:3].T # matrix containing 3 principal components

    X_projected=x_c@(Projection_Matrix)@Projection_Matrix.T    #projected data
    
    res=x_c-X_projected #residuals

    residuals=pd.DataFrame(res)#residuals into dataframe

    last_day=residuals.tail(1)#last day residuals

    maxValueIndex = last_day.idxmax(axis=1)#Index for highest residual

    minValueIndex=last_day.idxmin(axis=1)# Index for lower residual


    max_last_day=last_day.iloc[:,maxValueIndex].values #residual with highest day

    min_last_day=last_day.iloc[:,minValueIndex].values
    
    #next_seven_day
    x_c_df=pd.DataFrame(x_c)
    x_m_df=pd.DataFrame(x_m)
    last_day_x_c=x_c_df.tail(1)
    last_day_x_m=x_m_df.tail(1)
    #print(last_day_x_m.iloc[:,maxValueIndex].values)
    max_swap_res_substraction=(last_day_x_c.iloc[:,maxValueIndex].values-last_day_x_m.iloc[:,maxValueIndex].values)
    #we are expecting the substracition to be positive, meanign that we locked a high fixed rate and in 7 days, rate will come down
    #print(max_swap_res_substraction)
    min_swap_res_substraction=(last_day_x_c.iloc[:,minValueIndex].values-last_day_x_m.iloc[:,minValueIndex].values)
    #print(min_swap_res_substraction)
    ##find positive values(what we want) and negative values when reciving fixed
    neg_count0 = len(list(filter(lambda x: (x < 0), max_swap_res_substraction))) 
    pos_count0 = len(list(filter(lambda x: (x >= 0), max_swap_res_substraction )))
    pos_count_list0.append(pos_count0)  
    ##find negative values(what we want) and positive values when paying fixed
    neg_count1 = len(list(filter(lambda x: (x < 0), min_swap_res_substraction))) 
    pos_count1 = len(list(filter(lambda x: (x >= 0), min_swap_res_substraction )))
    neg_count_list.append(pos_count1)  
    if n>=dataset.shape[0]:
        break
print("Winning days (when we recive the fixed rate)%:",(np.sum(pos_count_list0))/n_loops)
print("Winning days (when we recive the paying rate)%:",(np.sum(neg_count_list))/n_loops)
 

     
## if residual large, it is expensive, receive the fixed rate
## if residual small, pay the rate


    

Winning days (when we recive the fixed rate)%: 0.6424965580541533
Winning days (when we recive the paying rate)%: 0.6424965580541533


In [18]:
#RUN PCA LOOP using residuals to predict if residuals go down next day
i=0
j=50
m=50
n=51
window_size=50
n_loops=dataset.shape[0]/window_size
pos_count_list_next_day=[]
while i <=dataset.shape[0]:
    x_c=dataset.iloc[i:j,0:12].values
    x_m=dataset.iloc[m:n,0:12].values
    j=j+50
    i=i+50
    m=m+50
    n=n+50
    covariance_matrix=np.cov(x_c.T)
    eigen_values,eigen_vectors=np.linalg.eig(covariance_matrix)    #eigenvalues/vectors
    
    Projection_Matrix=(eigen_vectors.T[:][:])[:3].T # matrix containing 3 principal components

    X_projected=x_c@(Projection_Matrix)@Projection_Matrix.T    #projected data
    
    res=x_c-X_projected #residuals
   

    residuals=pd.DataFrame(res)#residuals into dataframe
    
    last_day=residuals.tail(1)#last day residuals

    maxValueIndex = last_day.idxmax(axis=1)#Index for highest residual
    

    minValueIndex=last_day.idxmin(axis=1)# Index for lower residual
    
    max_last_day=last_day.iloc[:,maxValueIndex].values     #print value for the index with the highest residuals

    min_last_day=last_day.iloc[:,minValueIndex].values     #print value for the index with the lowest residuals
    mean_res=np.mean(res)#mean 
    std_res=np.std(res)#std
    std2_above= mean_res+2*std_res
    std2_below=mean_res-2*std_res
    if max_last_day>std2_above or min_last_day<std2_below:
        x_c_df=pd.DataFrame(x_c)

        x_m_df=pd.DataFrame(x_m)

        last_day_x_c=x_c_df.tail(1)

        last_day_x_m=x_m_df.tail(1)

        max_swap_res_substraction=(last_day_x_c.iloc[:,maxValueIndex].values-last_day_x_m.iloc[:,maxValueIndex].values)
      #we are expecting the substracition to be positive, meanign that we locked a high fixed rate and in 7 days, rate will come down

        min_swap_res_substraction=(last_day_x_c.iloc[:,minValueIndex].values-last_day_x_m.iloc[:,minValueIndex].values)
      ##find positive values(what we want) and negative values for max_swap_rate_subs
      

        neg_count = len(list(filter(lambda x: (x < 0), max_swap_res_substraction))) 
        pos_count = len(list(filter(lambda x: (x >= 0), max_swap_res_substraction )))

        pos_count_list_next_day.append(pos_count)  
    
    if n>=dataset.shape[0]:
        break
    
print("Winning days:",(np.sum(pos_count_list_next_day))/n_loops)
      

    
 

Winning days: 0.34419458467186786
